## **3.1 ทดลองซ้ำและทำการปรับพารามิเตอร์ให้มากกว่าใน lecture หรือลอง generate data ที่แตกต่างจากที่เรียน** ##

สร้างข้อมูลขึ้นมาเองหรือเรียกใช้ชุดข้อมูลที่มี

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


# Load datasets
data_10sample = pd.read_csv('C:/ml/Data/sin_noisy_10sample.csv')
data_20sample = pd.read_csv('C:/ml/Data/sin_noisy_20sample.csv')
data_40sample = pd.read_csv('C:/ml/Data/sin_noisy_40sample.csv')
data_80sample = pd.read_csv('C:/ml/Data/sin_noisy_80sample.csv')

# Combine datasets into a dictionary
data_dict = {
    10: data_10sample,
    20: data_20sample,
    40: data_40sample,
    80: data_80sample
}



C:\Users\premj\AppData\Local\Temp\ipykernel_48436\1198401806.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# Custom MSE function
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

#กำหนด degree ที่ต้องการ
degrees = [1,3,8,15,20]

results = []

#วนลูปผ่านแต่ละชุดข้อมูลใน data_dict
for n_samples, data in data_dict.items():
    X = data[['x']].values
    y = data['noisy_y'].values
    
    #วนลูปผ่านแต่ละ polynomial degree ใน degrees
    for degree in degrees:
        poly = PolynomialFeatures(degree)
        X_poly = poly.fit_transform(X)
        
        model = LinearRegression()
        model.fit(X_poly, y)
        
        # Predictions on the training set
        y_pred_train = model.predict(X_poly)
        rmse_train = np.sqrt(mean_squared_error(y, y_pred_train))
        
        # Predictions using cross-validation
        scores = cross_val_score(model, X_poly, y, scoring='neg_mean_squared_error', cv=10)
        rmse_cv = np.sqrt(-scores.mean())
        
        # Record the results
        results.append({
            'n_samples': n_samples,
            'degree': degree,
            'rmse_train': rmse_train,
            'rmse_cv': rmse_cv
        })

results_df = pd.DataFrame(results)

# Display the results
results_df = results_df.round(2)
results_df


,n_samples,degree,rmse_train,rmse_cv
0,10,1,0.57,0.75
1,10,3,0.23,0.43
2,10,8,0.06,19.04
3,10,15,0.00,42.66
4,10,20,0.00,58.57
5,20,1,0.43,0.55
6,20,3,0.27,0.38
7,20,8,0.19,0.84
8,20,15,0.09,3052.76
9,20,20,0.00,75133.48


In [3]:
# แสดง coefficients สำหรับ polynomial degree ต่าง ๆ
coefficients = {}

for n_samples, data in data_dict.items():
    X = data[['x']].values
    y = data['noisy_y'].values
    
    for degree in degrees:
        poly = PolynomialFeatures(degree)
        X_poly = poly.fit_transform(X)
        
        model = LinearRegression()
        model.fit(X_poly, y)
        
        coefficients[(n_samples, degree)] = model.coef_

# แสดง coefficients
for key, value in coefficients.items():
    print(f'n_samples: {key[0]}, degree: {key[1]}')
    print(value)

n_samples: 10, degree: 1
[0.         1.07222121]
n_samples: 10, degree: 3
[ 0.          2.77721639  0.52324301 -2.59796523]
n_samples: 10, degree: 8
[  0.           1.01530933  -5.10343645  20.00527424  31.28377247
 -67.33168913 -67.42238562  55.62458056  50.54873512]
n_samples: 10, degree: 15
[-6.01200426e-12  8.01006018e-01  2.57815813e+00  2.35519874e+01
 -4.27306461e+01 -7.87939303e+01  1.25875127e+02  4.42920453e+01
 -4.11658516e+01  5.80704269e+01 -7.58955402e+01  1.54358693e+01
 -3.56924253e+01 -3.51335297e+01  1.87063106e+01 -7.66524420e+01]
n_samples: 10, degree: 20
[ 1.04680752e-11  8.72421399e-01  2.52898877e+00  2.11249632e+01
 -4.10416941e+01 -6.16369896e+01  1.13474875e+02  1.71685419e+01
 -1.89278185e+01  3.82504848e+01 -6.40546506e+01  2.47533492e+01
 -5.22000755e+01  3.60594553e+00 -2.54395224e+01 -1.48852131e+01
 -6.77957568e-01 -2.85768338e+01  1.80355810e+01 -3.80197540e+01
  3.10628893e+01]
n_samples: 20, degree: 1
[0.         1.05476692]
n_samples: 20, degree: 3
[